# make a very simple LSTM model with keras

see how long this will take to run for 7t data with a not too complicated model and what kind of resources we are talking about - single run single subject..

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler
import pandas as pd
import nibabel as nb
import hrf_tools
#import hcp_utils as hcp
#from analysis import plot_59k_results

#import matplotlib.pyplot as plt


In [5]:
print(np.__version__)

1.19.5


In [6]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15254561423770763324
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10770692224
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4413532262424955830
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### load data

In [9]:
subject=100610
feature='rms'
n_movies=4
scaler = StandardScaler()

stim = ['tfMRI_MOVIE1_7T_AP','tfMRI_MOVIE2_7T_PA','tfMRI_MOVIE3_7T_PA','tfMRI_MOVIE4_7T_AP']
stim_feat = ['7T_MOVIE1_CC1_v2', '7T_MOVIE2_HO1_v2', '7T_MOVIE3_CC2_v2', '7T_MOVIE4_HO2_v2']
    
i=0
X = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_rms.npy')
X.shape#plt.plot(X[0:1000])

(9211, 1)

In [10]:
X  = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_as_embed.npy')

In [11]:
X.shape

(1918, 1024)

In [12]:
from scipy.signal import resample

In [13]:
X = resample(X, 921, axis=0) #resample to 1hz for now 

In [14]:
X_hrf = hrf_tools.apply_optimal_hrf_10hz(X,1)

In [15]:
#plt.plot(X[:,1])

NameError: name 'plt' is not defined

In [16]:
img = nb.load('../sourcedata/data/parcellations/parcellated/sub100610_tfMRI_MOVIE1_7T_AP.ptseries.nii')

In [17]:
Y = img.get_fdata()
Y.shape

(921, 360)

In [18]:
# im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
# img = nb.load(im_file)
# img_y = img.get_fdata()
# Y = img_y
#Y is time x brain
# Y.shape

In [19]:
#X is time x embed
X_hrf.shape

(921, 1024)

### design a simple(?) lstm model

In [20]:
# model = keras.Sequential(
#     [
#         layers.Dense(2, activation="relu", name="layer1"),
#         layers.Dense(3, activation="relu", name="layer2"),
#         layers.Dense(4, name="layer3"),
#     ]
# )
# # Call model on a test input
# x = tf.ones((3, 3))
# y = model(x)

In [22]:
# Batch size
batch_size = 64
time_steps = 10
input_units = 1024
input_shape = (time_steps,input_units)

model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
#model.add(layers.Embedding(input_dim=1024, output_dim=1024))
model.add(keras.Input(shape=input_shape))
# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(100))
#model.add(layers.UpSampling1D(size=10))
#model.add(layers.Dense(17049))
model.add(layers.Dense(200))
model.add(layers.Dense(300))

model.add(layers.Dense(360))



# Add a Dense layer with 10 units.
#model.add(layers.Dense(1721))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               450000    
_________________________________________________________________
dense_2 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               60300     
_________________________________________________________________
dense_4 (Dense)              (None, 360)               108360    
Total params: 638,860
Trainable params: 638,860
Non-trainable params: 0
_________________________________________________________________


In [23]:
#dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(x_train,0), np.expand_dims(y_train,0)))
dataset = tf.data.Dataset.from_tensor_slices((X_hrf, Y))

dataset.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(360,), dtype=tf.float64, name=None))

In [24]:
#https://keras.io/api/preprocessing/timeseries/
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_hrf, 
                                                     targets = Y, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [25]:
dataset

<BatchDataset shapes: ((None, None, 1024), (None, 360)), types: (tf.float32, tf.float64)>

In [26]:


# # Buffer size to shuffle the dataset
# # (TF data is designed to work with possibly infinite sequences,
# # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# # it maintains a buffer in which it shuffles elements).
# BUFFER_SIZE = 10000

# dataset = (
#     dataset
#     .shuffle(BUFFER_SIZE)
#     .batch(batch_size, drop_remainder=True)
#     .prefetch(tf.data.experimental.AUTOTUNE))

# dataset

In [27]:
#simple_rnn = layers.SimpleRNN(18, return_sequences=True, return_state=True)

In [28]:
#whole_sequence_output, final_state = simple_rnn(np.expand_dims(x_train,0))

In [29]:
model.compile(
    loss=keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    optimizer="adam",
    metrics=["accuracy", 'mean_squared_error'],
)

In [ ]:
history = model.fit(dataset,epochs = 1000)

In [33]:
model.save('../outputs/')


INFO:tensorflow:Assets written to: ../outputs/assets


INFO:tensorflow:Assets written to: ../outputs/assets


In [ ]:
model = keras.models.load_model('../outputs/saved_model.pb')

In [163]:
x_test_path = '../data/char_test.npy'
x_test = np.load(x_test_path)
x_test.shape

(198, 9)

In [164]:
y_test_path = '../data/brain_test.npy'
y_test = np.load(y_test_path)
#y_train = y_train[:,0:18]
y_test.shape

(198, 1721)

In [165]:
model.evaluate(x_test,y_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.9385 - accuracy: 0.0000e+00 - mean_squared_error: 0.9385


[0.938504695892334, 0.0, 0.938504695892334]